In [ ]:
!pip install timm

## imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import random
import matplotlib
import cv2

import timm
import torch
import torch.optim as optim
from torch.optim import Adam
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision

from tqdm.notebook import tqdm
import albumentations as A
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# Initialization
train_csv_loc= '../input/plantpathology288x288/train.csv' 
test_csv_loc='../input/plantpathology288x288/sample_submission.csv'
train_image_loc = '../input/plantpathology288x288/train'
test_image_loc = '../input/plantpathology288x288/test'

train_csv = pd.read_csv(train_csv_loc)
test_csv = pd.read_csv(test_csv_loc)

image_size = 288
model_name = 'resnet200d'
seed = 719
batch_size = 32
num_workers = 0

## Processing the dataset csv

In [ ]:
lbl_map = {'0': 'complex', '1': 'frog_eye_leaf_spot', '2': 'healthy', '3': 'powdery_mildew', '4': 'rust', '5': 'scab'}
inv_lbl_map = {'complex': 0, 'frog_eye_leaf_spot': 1, 'healthy': 2, 'powdery_mildew': 3, 'rust': 4, 'scab': 5}
data_labels = list(lbl_map.values())

def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['image'][i], mapping_csv['labels'][i]
        mapping[name] = tags.split(' ')
    return mapping

ds_mapping = create_file_mapping(train_csv)

def one_hot_encode(tags, mapping):
	# create empty vector
	encoding = np.zeros(len(mapping), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[mapping[tag]] = 1
	return encoding

def prepare_csv_dict(file_mapping, tag_mapping):
    ds_dict = {}
    for filename in file_mapping.keys():
        # get tags
        tags = file_mapping[filename]
        # one hot encode tags
        target = one_hot_encode(tags, tag_mapping)
        # store
        ds_dict[filename] = target

    created_csv = pd.DataFrame.from_dict(ds_dict, orient='index').reset_index()
    created_csv.columns = ['image'] + data_labels
    return created_csv

one_hot_csv = prepare_csv_dict(ds_mapping, inv_lbl_map)

In [ ]:
# GPU settings
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
class parseDataset(Dataset):
    def __init__(self, out_csv, image_loc, transform=None, test=False, aug=None):
        self.out_csv = out_csv
        self.image_loc = image_loc
        self.transform = transform
        self.test = test
        self.aug = aug

    def __len__(self):
        return len(self.out_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_loc,
                                self.out_csv.iloc[idx, 0])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = image.astype(np.uint8)

        if self.aug:
            image = self.aug(image=image)["image"]
            
        if self.transform:
            image = self.transform(image)
        
        if self.test:
            return image
        else:
            categories = self.out_csv.iloc[idx,1:]
            categories = np.array(categories)
            categories = categories.astype(np.uint8)
            sample = [image, categories]

            return sample

In [ ]:
data_transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize(
                     mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225],
     ),
    ])

aug_transform = A.Compose([
                A.RandomSizedCrop(min_max_height=(200, 270), height=288, width=288, p=0.5),
                A.PadIfNeeded(min_height=288, min_width=288, p=1),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.8), 
                A.RandomGamma(p=0.8)
])

## load dataset

In [ ]:
def prep_dataloader(data_csv, train_index, valid_index, image_loc):

    train_dataset = parseDataset(train_index, image_loc, transform = data_transform, aug=aug_transform)
    valid_dataset =  parseDataset(valid_index, image_loc, transform = data_transform)
    
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=2)
    valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle=True, num_workers=2)   
    
    return train_dataloader, valid_dataloader 

## define model

In [ ]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        
        # load pretrained weights
        self.model.load_state_dict(torch.load('../input/resnet200dpretrainedweights/resnet200d_ra2-bdba9bf9.pth'))
        
#         self.model.conv1[0].in_channels = 1
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
      
        self.fc = nn.Sequential(
                        nn.Linear(n_features, 256),
                        nn.Dropout(p=0.2),
                        nn.Linear(256, 6),
                    )
    
#         for param in self.model.parameters():
#             param.requires_grad = False

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [ ]:
net = ResNet200D(model_name = 'resnet200d')
net = net.to(device)
# print(net)

In [ ]:
!ls ../input/plantpathweightsresnet288x288

In [ ]:
LAST_WEIGHT_PATH = '../input/plantpathweightsresnet288x288/resnet200d_fold_0_epoch_50.pth'
net.load_state_dict(torch.load(LAST_WEIGHT_PATH))

## Training and Validation Loop

In [ ]:
def train_loop(train_dataloader, net, criterion, optimizer, device, ths):
    net.train()
    preds = []
    targets = []
    losses = []
    t = tqdm(train_dataloader, desc="Training: ")
    for images, labels in t:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = net(images)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        preds += [ outputs.sigmoid() ]
        targets += [ labels.detach().cpu() ]
        
        losses.append(loss.item())
        t.set_postfix_str("training loss: {:.3f}".format(loss.item()))
        
    preds = torch.cat(preds).detach().cpu().numpy()
    targets = torch.cat(targets).cpu().numpy() # undo batching
    out = f1_score(targets, preds > ths, average = 'macro')
    

    print(f"train F1: {out:.3f}")
    loss_train = np.mean(losses)
    return loss_train

In [ ]:
def valid_loop(valid_dataloader, net, criterion, optimizer, device, ths):
    net.eval()
    preds = []
    targets = []
    losses = []
    
    with torch.no_grad():
        t = tqdm(valid_dataloader, desc='Validating: ', colour="#ff7f50")
        for images, labels_ in t:
            images, labels_ = images.to(device), labels_.to(device)
            
            outputs = net(images)
            loss = criterion(outputs, labels_.float())
            
            preds += [ outputs.sigmoid() ] # **
            targets += [ labels_.detach().cpu() ]
            
            losses.append(loss.item())
            t.set_postfix_str('validation loss: {:.3f}'.format(loss.item()))
            
    preds = torch.cat(preds).cpu().numpy() # [ tensor1, tensor2, ... ] where tensor i = 2*11 matrix
    
    targets = torch.cat(targets).cpu().numpy() # undo batching
    
    out = f1_score(targets, preds > ths, average = 'macro')
    
    if debug:
        print(preds > ths)
        print(targets)

    print(f"val F1: {out:.3f}")
    
    loss_valid = np.mean(losses)
    return loss_valid

In [ ]:
epochs = 60
train = True
debug = False

best_loss = np.inf

criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(net.parameters(), lr=0.0001, weight_decay = 1e-6, amsgrad = False)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=6)

In [ ]:
split_frac = 0.8
fold = 0

In [ ]:
def run_training(net, device, in_csv, split_frac, epochs, criterion, optimizer, img_loc, best_loss):
    seed_everything(seed)

    in_csv = in_csv.sample(frac=1)
    train_index, valid_index = np.split(in_csv, [int(len(in_csv) * split_frac)])
    print('train set count: {} \nvalidation set count: {}'.format(len(train_index), len(valid_index)))

    for epoch in range(1, epochs + 1):
        print('Epoch: {}/{}'.format(epoch, epochs))
        
        # print LR
        for param_group in optimizer.param_groups:
            print("LR:", param_group['lr'])

        train_loader, valid_loader = prep_dataloader(in_csv, train_index, valid_index, img_loc)

        avg_loss = train_loop(train_loader, net, criterion, optimizer, device, 0.4)
        print("avg loss: {:.3f}".format(avg_loss))

        avg_val_loss = valid_loop(valid_loader, net, criterion, optimizer, device, 0.4)
        print("val loss: {:.3f}\n".format(avg_val_loss))
        scheduler.step(avg_val_loss)

        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(net.state_dict(), "/kaggle/working/{}_fold_{}_epoch_{}.pth".format(model_name, fold, epoch))

if train:
    run_training(net, device, one_hot_csv, split_frac, epochs, criterion, optimizer, train_image_loc, best_loss)

## testing
helper functions

In [ ]:
def get_labels(row, labels, ths=[0.5] * len(lbl_map)):
    try:
        row = [i for i, x in enumerate(row) if x > ths]
        row = [labels[str(i)] for i in row]
        row = 'healthy' if ('healthy' in row or len(row) == 0) else ' '.join(row)
    except:
        print(row)
    return row

In [ ]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((image_size, image_size)),
        transforms.Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ])

## testing for thresholds
(this part is run only if `train = False`) 
predictions are made on the entire dataset and the best thresholds for each class are recorded

In [ ]:
criterion = nn.BCEWithLogitsLoss()

def get_thresholds(in_csv, img_loc, device, net, criterion):
    net.eval()
    batch_size = 16

    # parse dataset and create dataloader
    test_dataset = parseDataset(in_csv, img_loc, transform = test_transform)
    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=2)

    preds_list = []
    labels_list = []

    # get output
    t = tqdm(test_dataloader, desc='testing: ', colour="#557f50")
    for images, labels in t:
        images = images.to(device)

        outputs = net(images)

        preds = outputs.sigmoid().detach().cpu().numpy()
        preds_list.append(preds)
        labels_list.append(labels)
    
    return np.concatenate(preds_list), np.concatenate(labels_list)
    
if not train:
    p_list, l_list = get_thresholds(one_hot_csv, train_image_loc, device, net, criterion)

    thresholds = np.arange(.01, 1., .01)
    scores = {}

    for threshold in thresholds:
        out_score = f1_score(l_list, p_list > threshold, average = None)
        scores[ threshold ] = list(out_score)
        
    ths_idxs = np.argmax(list(scores.values()), axis=0)
    best_ths = [thresholds[i] for i in ths_idxs]
    print(best_ths)